In [ ]:
import rasterio
import numpy as np

input_path = r"E:\Hyperspectral\DataOutput\Test\NEON_HSI_2021.tif"
output_path = r"E:\Hyperspectral\DataOutput\Test\NEON_HSI_2021_Scaled.tif"

with rasterio.open(input_path) as src:
    profile = src.profile
    height, width = src.height, src.width

    valid_bands = []
    scaled_bands = []

    print(f"Normalizing {src.count} bands and removing flat ones...")
    for i in range(1, src.count + 1):
        band = src.read(i).astype('float32')
        band_min = band.min()
        band_max = band.max()

        if band_max - band_min < 1e-6:
            print(f"Band {i}: Skipped (flat or invalid)")
            continue

        scaled_band = (band - band_min) / (band_max - band_min)
        scaled_bands.append(scaled_band)
        valid_bands.append(i)

    scaled_data = np.stack(scaled_bands).astype('float32')

    profile.update(dtype='float32', count=len(valid_bands))

with rasterio.open(output_path, 'w', **profile) as dst:
    dst.write(scaled_data)

print(f"Saved {len(valid_bands)} valid normalized bands to:\n{output_path}")
print(f"Skipped {src.count - len(valid_bands)} flat/invalid bands.")


Normalizing 426 bands and removing flat ones...


In [6]:
import rasterio
from rasterio import windows
import numpy as np

image_path = r"E:\Hyperspectral\DataOutput\Test\NEON_HSI_2021_Scaled.tif"
output_path = r"E:\Hyperspectral\DataOutput\Test\NEON_HSI_10Bands.tif"

# ========== Bands to Extract (1-based indexing) ==========
band_numbers = [59, 150, 211, 212, 213, 214, 228, 369, 371, 372]

with rasterio.open(image_path) as src:
    profile = src.profile
    profile.update(count=len(band_numbers)) 
    
    # Read selected bands
    selected_bands = []
    for b in band_numbers:
        band_data = src.read(b)
        selected_bands.append(band_data)
    
    selected_bands = np.stack(selected_bands)

    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(selected_bands)

print(f"Selected bands saved successfully to:", output_path)


Selected bands saved successfully to: E:\Hyperspectral\DataOutput\Test\NEON_HSI_10Bands.tif


In [7]:
import numpy as np
import rasterio
import joblib
from rasterio.features import shapes
from shapely.geometry import shape
import geopandas as gpd

test_image_path = r"E:\Hyperspectral\DataOutput\Test\NEON_HSI_10Bands.tif"
model_path = r"E:\Hyperspectral\DataOutput\rf_model.pkl"
test_output_path = r"E:\Hyperspectral\DataOutput\Test\RF_TestPrediction.tif"
test_shapefile_path = r"E:\Hyperspectral\DataOutput\Test\Test_Builtup.shp"

rf = joblib.load(model_path)
builtup_class_value = 1  

with rasterio.open(test_image_path) as src:
    img = src.read()  # Shape: (bands, rows, cols)
    meta = src.meta.copy()
    transform = src.transform
    crs = src.crs
    n_bands, n_rows, n_cols = img.shape

flat_pixels = img.reshape(n_bands, -1).T
nodata_mask = np.all(flat_pixels == 0, axis=1)

valid_pixels = flat_pixels[~nodata_mask]
predicted = rf.predict(valid_pixels)

# Fill full image
classified = np.full(flat_pixels.shape[0], -1, dtype=np.int16)
classified[~nodata_mask] = predicted
classified = classified.reshape(n_rows, n_cols)

meta.update({'count': 1, 'dtype': 'int16'})
with rasterio.open(test_output_path, 'w', **meta) as dst:
    dst.write(classified, 1)
print("Prediction GeoTIFF saved: {test_output_path}")

proba = rf.predict_proba(valid_pixels)
class_labels = rf.classes_
builtup_index = np.where(class_labels == builtup_class_value)[0][0]
builtup_proba = proba[:, builtup_index]
threshold_mask = builtup_proba > 0.5

thresholded_builtup = np.zeros(flat_pixels.shape[0], dtype=np.uint8)
thresholded_builtup[~nodata_mask] = threshold_mask.astype(np.uint8)
thresholded_builtup = thresholded_builtup.reshape(n_rows, n_cols)

results = (
    {'properties': {'class': 1}, 'geometry': shape(geom)}
    for geom, val in shapes(thresholded_builtup.astype(np.uint8), mask=(thresholded_builtup == 1), transform=transform)
)
gdf = gpd.GeoDataFrame.from_features(list(results), crs=crs)


gdf["area"] = gdf.geometry.area
min_area = 50 
gdf = gdf[gdf["area"] > min_area]


gdf.to_file(test_shapefile_path)
print(f"Built-up shapefile saved: {test_shapefile_path}")


✅ Prediction GeoTIFF saved: E:\Hyperspectral\DataOutput\Test\RF_TestPrediction.tif
✅ Built-up shapefile saved: E:\Hyperspectral\DataOutput\Test\Test_Builtup.shp


In [10]:
import geopandas as gpd
from shapely.geometry import Polygon
from shapely import minimum_rotated_rectangle


input_path = r"E:\Hyperspectral\DataOutput\Test\Test_Builtup_Final_.shp"
gdf = gpd.read_file(input_path)


if gdf.crs.is_geographic:
    gdf = gdf.to_crs(epsg=32612)  


gdf['geometry'] = gdf['geometry'].apply(lambda geom: minimum_rotated_rectangle(geom))


output_path = r"E:\Hyperspectral\DataOutput\Test\rectified_buildings.shp"
gdf.to_file(output_path)

print(f"Rectified building polygons saved to:", output_path)


✅ Rectified building polygons saved to: E:\Hyperspectral\DataOutput\Test\rectified_buildings.shp
